In [22]:
import h5py 

# data_path = "/home/binhng/Workspace/robocasa/robocasa/datasets/debug/CoffeeSetupMug.hdf5"
data_path = "/home/binhng/Workspace/robocasa/robocasa/datasets/regenerate/robocasa-30demos-5chosen-tasks-element/CoffeeSetupMug.hdf5"
data = h5py.File(data_path)

In [27]:
# data['data'].keys()
data['data']['demo_1500']['obs']['robot0_gripper_qpos']

<HDF5 dataset "robot0_gripper_qpos": shape (284, 2), type "<f8">

In [5]:
demo_id = 1688
# left_mask = data['data'][f'demo_{demo_id}']['obs']["robot0_agentview_left_segmentation_instance"][:]
# right_mask = data['data'][f'demo_{demo_id}']['obs']["robot0_agentview_right_segmentation_instance"][:]
# hand_mask = data['data'][f'demo_{demo_id}']['obs']["robot0_eye_in_hand_segmentation_instance"][:]

left_mask = data['data'][f'demo_{demo_id}']['obs']["robot0_agentview_left_segmentation_element"][:]
right_mask = data['data'][f'demo_{demo_id}']['obs']["robot0_agentview_right_segmentation_element"][:]
hand_mask = data['data'][f'demo_{demo_id}']['obs']["robot0_eye_in_hand_segmentation_element"][:]
left_mask.shape, right_mask.shape, hand_mask.shape

((267, 256, 256, 1), (267, 256, 256, 1), (267, 256, 256, 1))

In [6]:
data['data'][f'demo_{demo_id}']['obs'].keys()

<KeysViewHDF5 ['obj_pos', 'obj_quat', 'obj_to_robot0_eef_pos', 'obj_to_robot0_eef_quat', 'object-state', 'robot0_agentview_left_depth', 'robot0_agentview_left_image', 'robot0_agentview_left_segmentation_element', 'robot0_agentview_right_depth', 'robot0_agentview_right_image', 'robot0_agentview_right_segmentation_element', 'robot0_base_pos', 'robot0_base_quat', 'robot0_base_to_eef_pos', 'robot0_base_to_eef_quat', 'robot0_base_to_eef_quat_site', 'robot0_eef_pos', 'robot0_eef_quat', 'robot0_eef_quat_site', 'robot0_eye_in_hand_depth', 'robot0_eye_in_hand_image', 'robot0_eye_in_hand_segmentation_element', 'robot0_gripper_qpos', 'robot0_gripper_qvel', 'robot0_joint_acc', 'robot0_joint_pos', 'robot0_joint_pos_cos', 'robot0_joint_pos_sin', 'robot0_joint_vel', 'robot0_proprio-state']>

In [7]:
left_image = data['data'][f'demo_{demo_id}']['obs']["robot0_agentview_left_image"][:]
right_image = data['data'][f'demo_{demo_id}']['obs']["robot0_agentview_right_image"][:]
hand_image = data['data'][f'demo_{demo_id}']['obs']["robot0_eye_in_hand_image"][:]

In [8]:
# json.load('/home/binhng/Workspace/robocasa/robocasa/ooi_anno/CoffeeSetupMug/demo1688_left_element_ooi.json')

In [9]:
import json
json_ooi_dir = '/home/binhng/Workspace/robocasa/robocasa/datasets/regenerate/robocasa-30demos-5chosen-tasks-element/ooi_anno/CoffeeSetupMug/'
ooi_files = [
    f"demo{demo_id}_left_element_ooi.json",
    f"demo{demo_id}_right_element_ooi.json",
    f"demo{demo_id}_hand_element_ooi.json"
]

all_ooi_ids = []
for file in ooi_files:
    
    with open(json_ooi_dir+file, 'r', encoding='utf-8') as f:
        all_ooi_ids += json.load(f)  # parse JSON content from file

all_ooi_ids = list(set(all_ooi_ids))
all_ooi_ids

[743, 717, 719, 720, 723, 62]

In [ ]:
import numpy as np 
from typing import List

In [ ]:
def 

In [ ]:

def apply_mask_and_per_frame_bbox_keep_region(
    mask: np.ndarray,              # (T,H,W) or (T,H,W,1) integer IDs
    rgb: np.ndarray,               # (T,H,W,3) RGB uint8/float
    ids_of_interest: List[int],
    pad: int = 0
) -> np.ndarray:
    """
    For each frame t, compute a tight bbox around all pixels whose mask IDs are in ids_of_interest,
    expand bbox by `pad` pixels, and keep ALL RGB pixels inside that bbox; set outside to 0.

    Returns: out video (T,H,W,3), same dtype as `rgb`.
    """
    # Validate and squeeze channel
    assert mask.ndim in (3, 4), "mask must be (T,H,W) or (T,H,W,1)"
    assert rgb.ndim == 4 and rgb.shape[-1] == 3, "rgb must be (T,H,W,3)"
    if mask.ndim == 4:
        assert mask.shape[-1] == 1, "mask last dimension must be 1"
        mask = mask[..., 0]  # (T,H,W)

    T, H, W = mask.shape
    assert rgb.shape[:3] == (T, H, W), f"rgb shape {rgb.shape} incompatible with mask {mask.shape}"

    out = np.zeros_like(rgb)
    ids = np.array(ids_of_interest, dtype=mask.dtype)

    for t in range(T):
        hits = np.isin(mask[t], ids)  # (H,W) boolean of interested IDs in this frame
        ys, xs = np.where(hits)
        if ys.size == 0:
            # No interested IDs this frame -> leave frame black
            continue

        # Tight bbox
        y0, y1 = int(ys.min()), int(ys.max())
        x0, x1 = int(xs.min()), int(xs.max())

        # Expand by pad and clamp to bounds (inclusive bbox)
        y0 = max(0, y0 - pad); y1 = min(H - 1, y1 + pad)
        x0 = max(0, x0 - pad); x1 = min(W - 1, x1 + pad)

        # Keep ALL pixels in bbox (no ID filtering inside bbox)
        out[t, y0:y1+1, x0:x1+1, :] = np.ones_like(rgb[t, y0:y1+1, x0:x1+1, :])*255
        # print(out.shape)

    return out


In [18]:
import imageio.v2 as imageio
import numpy as np
import cv2
from typing import List, Tuple, Optional

def compute_union_bbox(mask_3d: np.ndarray, ids_of_interest: List[int], pad: int = 5) -> Optional[Tuple[int,int,int,int]]:

    assert mask_3d.ndim in (3, 4), f"Expected (T,H,W) or (T,H,W,1), got {mask_3d.shape}"
    if mask_3d.ndim == 4:
        assert mask_3d.shape[-1] == 1, f"Last dim must be 1 for (T,H,W,1), got {mask_3d.shape}"
        mask_3d = mask_3d[..., 0]  # (T,H,W)

    T, H, W = mask_3d.shape
    ids = np.array(ids_of_interest, dtype=mask_3d.dtype)

    # Union across time of the pixels of interest
    hits = np.isin(mask_3d, ids)  # (T,H,W)
    hits_any = hits.any(axis=0)   # (H,W)

    ys, xs = np.where(hits_any)
    if ys.size == 0:
        return None

    y0, y1 = int(ys.min()), int(ys.max())
    x0, x1 = int(xs.min()), int(xs.max())

    # Expand by pad, clamp to bounds (inclusive bbox)
    y0 = max(0, y0 - pad); y1 = min(H - 1, y1 + pad)
    x0 = max(0, x0 - pad); x1 = min(W - 1, x1 + pad)

    return (x0, y0, x1, y1)

def apply_mask_and_bbox(
    mask: np.ndarray, rgb: np.ndarray, ids_of_interest: List[int], pad: int = 5, per_frame_bbox: bool = False
) -> np.ndarray:

    assert mask.ndim in (3, 4), "mask must be (T,H,W) or (T,H,W,1)"
    assert rgb.ndim == 4 and rgb.shape[-1] == 3, "rgb must be (T,H,W,3)"
    if mask.ndim == 4:
        assert mask.shape[-1] == 1, "mask last dimension must be 1"
        mask = mask[..., 0]

    T, H, W = mask.shape
    assert rgb.shape[:3] == (T, H, W), f"rgb shape {rgb.shape} incompatible with mask {mask.shape}"

    # Prepare output
    out = np.zeros_like(rgb)

    ids = np.array(ids_of_interest, dtype=mask.dtype)

    if not per_frame_bbox:
        # Single global bbox across time
        bbox = compute_union_bbox(mask, ids_of_interest, pad=pad)
        if bbox is None:
            # No hits: return zeros
            return out
        x0, y0, x1, y1 = bbox

        # Create a (H,W) mask of pixels-of-interest (union across IDs) per frame
        for t in range(T):
            roi_mask = np.isin(mask[t], ids)  # (H,W) boolean
            # Also constrain to bbox region
            frame_keep = np.zeros((H, W), dtype=bool)
            frame_keep[y0:y1+1, x0:x1+1] = roi_mask[y0:y1+1, x0:x1+1]

            # Apply to RGB
            out[t][frame_keep] = rgb[t][frame_keep]
    else:
        # Per-frame bbox (each frame gets its own tight bbox around its hits)
        for t in range(T):
            hits = np.isin(mask[t], ids)  # (H,W)
            ys, xs = np.where(hits)
            if ys.size == 0:
                continue
            y0, y1 = int(ys.min()), int(ys.max())
            x0, x1 = int(xs.min()), int(xs.max())
            # pad and clamp
            y0 = max(0, y0 - pad); y1 = min(H - 1, y1 + pad)
            x0 = max(0, x0 - pad); x1 = min(W - 1, x1 + pad)

            frame_keep = np.zeros((H, W), dtype=bool)
            frame_keep[y0:y1+1, x0:x1+1] = hits[y0:y1+1, x0:x1+1]
            out[t][frame_keep] = rgb[t][frame_keep]

    return out



import numpy as np
from typing import List, Tuple, Optional

def compute_union_bbox2(mask_3d: np.ndarray, ids_of_interest: List[int], pad: int = 5) -> Optional[Tuple[int,int,int,int]]:
    """
    Compute a tight bbox over all frames that includes any pixel whose ID is in ids_of_interest,
    then expand by `pad` pixels along both axes. Returns (x0, y0, x1, y1) inclusive.
    If no pixels match, returns None.
    """
    assert mask_3d.ndim in (3, 4), f"Expected (T,H,W) or (T,H,W,1), got {mask_3d.shape}"
    if mask_3d.ndim == 4:
        assert mask_3d.shape[-1] == 1, f"Last dim must be 1 for (T,H,W,1), got {mask_3d.shape}"
        mask_3d = mask_3d[..., 0]  # (T,H,W)

    T, H, W = mask_3d.shape
    ids = np.array(ids_of_interest, dtype=mask_3d.dtype)

    hits = np.isin(mask_3d, ids)   # (T,H,W)
    hits_any = hits.any(axis=0)    # (H,W)

    ys, xs = np.where(hits_any)
    if ys.size == 0:
        return None

    y0, y1 = int(ys.min()), int(ys.max())
    x0, x1 = int(xs.min()), int(xs.max())

    # Expand by pad and clamp (inclusive bbox)
    y0 = max(0, y0 - pad); y1 = min(H - 1, y1 + pad)
    x0 = max(0, x0 - pad); x1 = min(W - 1, x1 + pad)
    return (x0, y0, x1, y1)


def apply_mask_and_bbox_keep_region2(
    mask: np.ndarray,
    rgb: np.ndarray,
    ids_of_interest: List[int],
    pad: int = 5,
    per_frame_bbox: bool = False
) -> np.ndarray:
    """
    Produce (T,H,W,3) where ALL pixels inside the chosen bbox are kept from `rgb`,
    and ALL pixels outside the bbox are set to 0.

    - If per_frame_bbox=False: use a single global bbox computed over all frames.
    - If per_frame_bbox=True : compute a tight bbox per frame.
    """
    # Validate shapes and squeeze channel
    assert mask.ndim in (3, 4), "mask must be (T,H,W) or (T,H,W,1)"
    assert rgb.ndim == 4 and rgb.shape[-1] == 3, "rgb must be (T,H,W,3)"
    if mask.ndim == 4:
        assert mask.shape[-1] == 1, "mask last dimension must be 1"
        mask = mask[..., 0]

    T, H, W = mask.shape
    assert rgb.shape[:3] == (T, H, W), f"rgb shape {rgb.shape} incompatible with mask {mask.shape}"

    out = np.zeros_like(rgb)

    if not per_frame_bbox:
        # Single global bbox
        bbox = compute_union_bbox2(mask, ids_of_interest, pad=pad)
        if bbox is None:
            # No interested IDs present -> all black
            return out
        x0, y0, x1, y1 = bbox

        # Keep ALL pixels inside the bbox (no ID filtering)
        for t in range(T):
            out[t, y0:y1+1, x0:x1+1, :] = rgb[t, y0:y1+1, x0:x1+1, :]

    else:
        # Per-frame bbox
        ids = np.array(ids_of_interest, dtype=mask.dtype)
        for t in range(T):
            hits = np.isin(mask[t], ids)
            ys, xs = np.where(hits)
            if ys.size == 0:
                # No hits in this frame -> leave all black
                continue

            y0, y1 = int(ys.min()), int(ys.max())
            x0, x1 = int(xs.min()), int(xs.max())
            y0 = max(0, y0 - pad); y1 = min(H - 1, y1 + pad)
            x0 = max(0, x0 - pad); x1 = min(W - 1, x1 + pad)

            # Keep ALL pixels inside this frame's bbox
            out[t, y0:y1+1, x0:x1+1, :] = rgb[t, y0:y1+1, x0:x1+1, :]

    return out




def write_video_mp4_imageio(frames_rgb: np.ndarray, path: str, fps: int = 30, quality: int = 8):
    """
    Write (T,H,W,3) RGB uint8 frames to an MP4 file using imageio (ffmpeg backend).
    - frames_rgb: np.ndarray of shape (T,H,W,3), dtype=uint8
    - path: output .mp4 file path
    - fps: frames per second
    - quality: ffmpeg 'quality' (lower is better; typical 5-8). Ignored for some codecs.
    """
    assert frames_rgb.ndim == 4 and frames_rgb.shape[-1] == 3, "frames must be (T,H,W,3)"
    if frames_rgb.dtype != np.uint8:
        frames_rgb = np.clip(frames_rgb, 0, 255).astype(np.uint8)

    # 'ffmpeg' writer with h264 codec (widely compatible)
    # You can switch to 'libx264' or 'libx265' depending on your ffmpeg build.
    writer = imageio.get_writer(
        path,
        fps=fps,
        # codec='libx264',
        # quality=quality,
        # format='FFMPEG',               # ensure ffmpeg backend
        # macro_block_size=None          # allow non-multiple-of-16 sizes (e.g., 256x256)
    )
    try:
        for frame in frames_rgb:
            writer.append_data(frame)  # assumes RGB uint8
    finally:
        writer.close()
        

def apply_mask_and_per_frame_bbox_keep_region(
    mask: np.ndarray,              # (T,H,W) or (T,H,W,1) integer IDs
    rgb: np.ndarray,               # (T,H,W,3) RGB uint8/float
    ids_of_interest: List[int],
    pad: int = 0
) -> np.ndarray:
    """
    For each frame t, compute a tight bbox around all pixels whose mask IDs are in ids_of_interest,
    expand bbox by `pad` pixels, and keep ALL RGB pixels inside that bbox; set outside to 0.

    Returns: out video (T,H,W,3), same dtype as `rgb`.
    """
    # Validate and squeeze channel
    assert mask.ndim in (3, 4), "mask must be (T,H,W) or (T,H,W,1)"
    assert rgb.ndim == 4 and rgb.shape[-1] == 3, "rgb must be (T,H,W,3)"
    if mask.ndim == 4:
        assert mask.shape[-1] == 1, "mask last dimension must be 1"
        mask = mask[..., 0]  # (T,H,W)

    T, H, W = mask.shape
    assert rgb.shape[:3] == (T, H, W), f"rgb shape {rgb.shape} incompatible with mask {mask.shape}"

    out = np.zeros_like(rgb)
    ids = np.array(ids_of_interest, dtype=mask.dtype)

    for t in range(T):
        hits = np.isin(mask[t], ids)  # (H,W) boolean of interested IDs in this frame
        ys, xs = np.where(hits)
        if ys.size == 0:
            # No interested IDs this frame -> leave frame black
            continue

        # Tight bbox
        y0, y1 = int(ys.min()), int(ys.max())
        x0, x1 = int(xs.min()), int(xs.max())

        # Expand by pad and clamp to bounds (inclusive bbox)
        y0 = max(0, y0 - pad); y1 = min(H - 1, y1 + pad)
        x0 = max(0, x0 - pad); x1 = min(W - 1, x1 + pad)

        # Keep ALL pixels in bbox (no ID filtering inside bbox)
        out[t, y0:y1+1, x0:x1+1, :] = np.ones_like(rgb[t, y0:y1+1, x0:x1+1, :])*255
        print(out.shape)

    return out



In [19]:
# left_ooi_rgb = apply_mask_and_bbox(left_mask, left_image, all_ooi_ids, pad=5)
# write_video_mp4_imageio(left_ooi_rgb, "./test_left.mp4")

In [20]:
# left_ooi_rgb = apply_mask_and_bbox_keep_region2(left_mask, left_image, all_ooi_ids, pad=5)
# write_video_mp4_imageio(left_ooi_rgb, "./test_left2.mp4")
# right_ooi_rgb = apply_mask_and_bbox_keep_region2(right_mask, right_image, all_ooi_ids, pad=5)
# write_video_mp4_imageio(right_ooi_rgb, "./test_right2.mp4")
# hand_ooi_rgb = apply_mask_and_bbox_keep_region2(hand_mask, hand_image, all_ooi_ids, pad=5)
# write_video_mp4_imageio(hand_ooi_rgb, "./test_hand2.mp4")

In [21]:
left_ooi_rgb = apply_mask_and_per_frame_bbox_keep_region(left_mask, left_image, all_ooi_ids, pad=5)
write_video_mp4_imageio(left_ooi_rgb, "./test_left4.mp4")
right_ooi_rgb = apply_mask_and_per_frame_bbox_keep_region(right_mask, right_image, all_ooi_ids, pad=5)
write_video_mp4_imageio(right_ooi_rgb, "./test_right4.mp4")
hand_ooi_rgb = apply_mask_and_per_frame_bbox_keep_region(hand_mask, hand_image, all_ooi_ids, pad=5)
write_video_mp4_imageio(hand_ooi_rgb, "./test_hand4.mp4")

(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 2

[libopenh264 @ 0x3b4cd8c0] [OpenH264] this = 0x0x3b4cd850, Warning:layerId(0) doesn't support profile(578), change to UNSPECIFIC profile
[libopenh264 @ 0x3b4cd8c0] [OpenH264] this = 0x0x3b4cd850, Warning:bEnableFrameSkip = 0,bitrate can't be controlled for RC_QUALITY_MODE,RC_BITRATE_MODE and RC_TIMESTAMP_MODE without enabling skip frame.
[libopenh264 @ 0x11bc08c0] [OpenH264] this = 0x0x11bc0850, Warning:layerId(0) doesn't support profile(578), change to UNSPECIFIC profile
[libopenh264 @ 0x11bc08c0] [OpenH264] this = 0x0x11bc0850, Warning:bEnableFrameSkip = 0,bitrate can't be controlled for RC_QUALITY_MODE,RC_BITRATE_MODE and RC_TIMESTAMP_MODE without enabling skip frame.


(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 256, 3)
(267, 256, 2

[libopenh264 @ 0xa3868c0] [OpenH264] this = 0x0xa386850, Warning:layerId(0) doesn't support profile(578), change to UNSPECIFIC profile
[libopenh264 @ 0xa3868c0] [OpenH264] this = 0x0xa386850, Warning:bEnableFrameSkip = 0,bitrate can't be controlled for RC_QUALITY_MODE,RC_BITRATE_MODE and RC_TIMESTAMP_MODE without enabling skip frame.


In [ ]:
right_ooi_rgb = apply_mask_and_bbox(right_mask, right_image, all_ooi_ids, pad=5)
write_video_mp4_imageio(right_ooi_rgb, "./test_right.mp4")

[libopenh264 @ 0x30c7e8c0] [OpenH264] this = 0x0x30c7e850, Warning:layerId(0) doesn't support profile(578), change to UNSPECIFIC profile
[libopenh264 @ 0x30c7e8c0] [OpenH264] this = 0x0x30c7e850, Warning:bEnableFrameSkip = 0,bitrate can't be controlled for RC_QUALITY_MODE,RC_BITRATE_MODE and RC_TIMESTAMP_MODE without enabling skip frame.


In [151]:
hand_ooi_rgb = apply_mask_and_bbox(hand_mask, hand_image, all_ooi_ids, pad=5)
write_video_mp4_imageio(hand_ooi_rgb, "./test_hand.mp4")

[libopenh264 @ 0x2f68f8c0] [OpenH264] this = 0x0x2f68f850, Warning:layerId(0) doesn't support profile(578), change to UNSPECIFIC profile
[libopenh264 @ 0x2f68f8c0] [OpenH264] this = 0x0x2f68f850, Warning:bEnableFrameSkip = 0,bitrate can't be controlled for RC_QUALITY_MODE,RC_BITRATE_MODE and RC_TIMESTAMP_MODE without enabling skip frame.


In [ ]:

    # Example: given mask (T,256,256,1) and rgb (T,256,256,3)
    # mask = ...  # np.ndarray
    # rgb  = ...  # np.ndarray
    # ids_of_interest = [5, 12, 447]  # example IDs

    # Convert RGB input to uint8 if needed (expected for video writing)
    # rgb_uint8 = np.asarray(np.clip(rgb, 0, 255), dtype=np.uint8)

    # Build masked video using a single global bbox (+5 px pad)
    # out_rgb = apply_mask_and_bbox(mask, rgb_uint8, ids_of_interest, pad=5, per_frame_bbox=False)

    # If your `rgb` is RGB, convert to BGR for OpenCV video writing:
    # out_bgr = cv2.cvtColor(out_rgb, cv2.COLOR_RGB2BGR)

    # write_video_mp4(out_bgr, "masked_ooi.mp4", fps=30)
    # pass
